In [1]:
"""
--- Day 20: Pulse Propagation ---

With your help, the Elves manage to find the right parts and fix all of the machines. Now, they just need to 
send the command to boot up the machines and get the sand flowing again.

The machines are far apart and wired together with long cables. The cables don't connect to the machines 
directly, but rather to communication modules attached to the machines that perform various initialization tasks 
and also act as communication relays.

Modules communicate using pulses. Each pulse is either a high pulse or a low pulse. When a module sends a pulse, 
it sends that type of pulse to each module in its list of destination modules.

There are several different types of modules:

Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high 
pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between 
on and off. If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected 
input modules; they initially default to remembering a low pulse for each input. When a pulse is received, the 
conjunction module first updates its memory for that input. Then, if it remembers high pulses for all inputs, it 
sends a low pulse; otherwise, it sends a high pulse.

There is a single broadcast module (named broadcaster). When it receives a pulse, it sends the same pulse to all 
of its destination modules.

Here at Desert Machine Headquarters, there is a module with a single button on it called, aptly, the button 
module. When you push the button, a single low pulse is sent directly to the broadcaster module.

After pushing the button, you must wait until all pulses have been delivered and fully handled before pushing it 
again. Never push the button if modules are still processing pulses.

Pulses are always processed in the order they are sent. So, if a pulse is sent to modules a, b, and c, and then 
module a processes its pulse and sends more pulses, the pulses sent to modules b and c would be handled first.

The module configuration (your puzzle input) lists each module. The name of the module is preceded by a symbol 
identifying its type, if any. The name is then followed by an arrow and a list of its destination modules. 
For example:
    broadcaster -> a, b, c
    %a -> b
    %b -> c
    %c -> inv
    &inv -> a
In this module configuration, the broadcaster has three destination modules named a, b, and c. Each of these 
modules is a flip-flop module (as indicated by the % prefix). a outputs to b which outputs to c which outputs to 
another module named inv. inv is a conjunction module (as indicated by the & prefix) which, because it has only 
one input, acts like an inverter (it sends the opposite of the pulse type it receives); it outputs to a.

By pushing the button once, the following pulses are sent:
    button -low-> broadcaster
    broadcaster -low-> a
    broadcaster -low-> b
    broadcaster -low-> c
    a -high-> b
    b -high-> c
    c -high-> inv
    inv -low-> a
    a -low-> b
    b -low-> c
    c -low-> inv
    inv -high-> a
After this sequence, the flip-flop modules all end up off, so pushing the button again repeats the same sequence.

Here's a more interesting example:
    broadcaster -> a
    %a -> inv, con
    &inv -> b
    %b -> con
    &con -> output
This module configuration includes the broadcaster, two flip-flops (named a and b), a single-input conjunction 
module (inv), a multi-input conjunction module (con), and an untyped module named output (for testing purposes). 
The multi-input conjunction module con watches the two flip-flop modules and, if they're both on, sends a low 
pulse to the output module. Here's what happens if you push the button once, twice, three times, and four times:

   button -low-> broadcaster   button -low-> broadcaster   button -low-> broadcaster   button -low-> broadcaster
   broadcaster -low-> a        broadcaster -low-> a        broadcaster -low-> a        button -low-> broadcaster
   a -high-> inv               a -low-> inv                a -high-> inv               broadcaster -low-> a
   a -high-> con               a -low-> con                a -high-> con               a -low-> inv
   inv -low-> b                inv -high-> b               inv -low-> b                a -low-> con
   con -high-> output          con -high-> output          con -low-> output           inv -high-> b
   b -high-> con                                           b -low-> con                con -high-> output
   con -low-> output                                       con -high-> output          
 1)Both flip-flops turn on    2)Flip-flop a turns off!   3)This time, flip-flop a     4)This completes the cycle:
 and a low pulse is sent to   Now, con remembers a low   turns on, then flip-flop b   a turns off, causing con to 
 output! However, now that    pulse from module a, and   turns off. However, before   remember only low pulses 
 both flip-flops are on and   so itsends only a high     b can turn off, the pulse    and restoring all modules 
 con remembers a high pulse   pulse to output.           sent to con is handled       to their original states.
 from each of its two inputs                             first, so it briefly  
                                                         remembers all high pulses for 
                                                         its inputs and sends a low 
                                                         pulse to output. After that, 
                                                         flip-flop b turns off, which 
                                                         causes con to update its state 
                                                         and send a high pulse to output.   

To get the cables warmed up, the Elves have pushed the button 1000 times. How many pulses got sent as a result 
(including the pulses sent by the button itself)?

In the first example, the same thing happens every time the button is pushed: 8 low pulses and 4 high pulses are 
sent. So, after pushing the button 1000 times, 8000 low pulses and 4000 high pulses are sent. Multiplying these 
together gives 32000000.

In the second example, after pushing the button 1000 times, 4250 low pulses and 2750 high pulses are sent. 
Multiplying these together gives 11687500.

Consult your module configuration; determine the number of low pulses and high pulses that would be sent after 
pushing the button 1000 times, waiting for all pulses to be fully handled after each push of the button. 
What do you get if you multiply the total number of low pulses sent by the total number of high pulses sent?

"""

import re

from typing import List, Tuple
from math import prod


class Module:
    def __init__(self, kind: str, name: str, dest: str):
        self.name = name
        self.relay = dest.split(", ")
        self.mem = 0 if kind == "%" else {}
        self.kind = kind
        
    def send(self, o: str, pulse: int) -> List[Tuple[str, int, str]]:
        if self.kind == "&":
            self.mem[o] = pulse
            sending = int(sum(self.mem.values()) != len(self.mem))
            return [(self.name, sending, i) for i in self.relay]
        elif not pulse:
            self.mem = not self.mem
            return [(self.name, int(self.mem), i) for i in self.relay]
        return []

        
initial = []
modules = {}
            
with open("input.txt") as f:
    for x in f.readlines():
        kind, name, dest = re.match(r"([%&]?)(\w+) -> (.*)", x).groups()
        if name == "broadcaster":
            initial = dest.split(", ")
            continue
        modules[name] = Module(kind, name, dest)
            
for name, module in modules.items():
    for r in module.relay:
        if r in modules and modules[r].kind == "&":
            modules[r].mem[name] = False

pulses = [0, 0]

for _ in range(1_000):
    pulses[0] += 1
    queue = [("O", 0, d) for d in initial]
    while queue:
        orig, pulse, dest = queue.pop(0)
        pulses[pulse] += 1
        if dest in modules:
            queue.extend(modules[dest].send(orig, pulse))
        
print(prod(pulses))
    
# total product is 919383692

919383692


In [2]:
"""
--- Part Two ---

The final machine responsible for moving the sand down to Island Island has a module attached named rx. 
The machine turns on when a single low pulse is sent to rx.

Reset all modules to their default states. Waiting for all pulses to be fully handled after each button press, 
what is the fewest number of button presses required to deliver a single low pulse to the module named rx?

"""

import re

from typing import List, Tuple, Dict
from math import lcm


class Module:
    def __init__(self, kind: str, name: str, dest: str):
        self.name = name
        self.relay = dest.split(", ")
        self.mem = 0 if kind == "%" else {}
        self.kind = kind
        
    def send(self, o: str, pulse: int) -> List[Tuple[str, int, str]]:
        if self.kind == "&":
            self.mem[o] = pulse
            sending = int(sum(self.mem.values()) != len(self.mem))
            return [(self.name, sending, i) for i in self.relay]
        elif not pulse:
            self.mem = not self.mem
            return [(self.name, int(self.mem), i) for i in self.relay]
        return []


def begin(initial: List[str], modules: Dict[str, Module]) -> int:
    c = 0
    trigger, = [name for name, m in modules.items() if "rx" in m.relay]
    requirements = {name: None for name, m in modules.items() if trigger in m.relay}
    
    while not all(requirements.values()):
        c += 1
        queue = [("O", 0, d) for d in initial]

        while queue:
            orig, pulse, dest = queue.pop(0)
            if orig in requirements and requirements[orig] is None and pulse == 1:
                requirements[orig] = c
            if dest in modules:
                queue.extend(modules[dest].send(orig, pulse))
                
    return lcm(*requirements.values())
    

initial = []
modules = {}
            
with open("input2.txt") as f:
    for x in f.readlines():
        kind, name, dest = re.match(r"([%&]?)(\w+) -> (.*)", x).groups()
        if name == "broadcaster":
            initial = dest.split(", ")
            continue
        modules[name] = Module(kind, name, dest)
            
for name, module in modules.items():
    for r in module.relay:
        if r in modules and modules[r].kind == "&":
            modules[r].mem[name] = False

print(begin(initial, modules))
    
# minimum is 247702167614647

247702167614647
